In [12]:
import csv
import numpy as np 
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from time import time 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Dropout, Embedding
from tensorflow.keras.layers import Bidirectional 
from sklearn.metrics import confusion_matrix, f1_score 

In [13]:
# 2번 블럭
# 파라미터 설정
MY_VOCAB = 5000   # 내가 사용할 단어 수, 인기도 기준
MY_EMBED = 64     # 임베딩 차원
MY_HIDDEN = 100   # RNN 셀의 규모
MY_LEN = 200      # 기사의 길이

MY_SPLIT = 0.8    # 학습용 데이터의 비율
MY_SAMPLE = 121   # 샘플 기사
MY_EPOCH = 10    # 반복 학습 수


In [14]:
#제외어 (Stopword) 설정 

nltk.download('stopwords')
MY_STOP = set(nltk.corpus.stopwords.words('english'))

print('제외어:')
print(MY_STOP)
print('제외어 갯수:', len(MY_STOP))


제외어:
{'doing', 'until', 'itself', 'so', 'because', 'by', 'why', 't', 'couldn', 'hadn', 'on', 'other', 'yourself', 'be', "mightn't", 'above', "you're", 'and', 'her', 'wasn', 'the', "couldn't", 'about', 'mightn', 'theirs', 'has', 'had', "it's", 'for', 'as', 'over', 'we', 'very', 'aren', 're', 'mustn', 'how', 'can', 'that', 'once', 'just', 'o', 'too', 'were', 'while', 'out', 'they', 'having', 'these', 'his', 'when', 'them', 'in', "hadn't", 'what', "you'd", 'who', 'here', 'herself', 'if', 'it', 'being', 'all', 'each', 'an', "needn't", 'against', 'at', 'further', 'yourselves', "doesn't", "you'll", 'than', 'didn', "weren't", 'more', 'wouldn', 'no', 've', 'any', 'those', 'are', "didn't", 'was', 'between', 'only', "won't", 'shouldn', 'ain', 'under', 'does', "shouldn't", "wasn't", "should've", 'whom', 'our', "haven't", 'which', 'most', 'am', 'myself', 'now', 'do', 'hasn', 'yours', "isn't", 'will', 'ma', 'down', 'to', "hasn't", 'hers', 'after', 'ourselves', 'own', 'shan', 'd', 'then', 'did', 'ha

In [15]:
# 4번 블럭
# 데이터 보관 창고
original = []
articles = []
labels = []

print(type(original))

<class 'list'>


In [16]:
with open('bbc-text.csv', 'r') as file:
    

    # 컬럼 이름 읽기
    reader = csv.reader(file)
    header = next(reader)
    print(header)

    # 기사 한줄씩 처리
    for row in reader:
        labels.append(row[0])
        original.append(row[1])

        # 제외어 삭제하기
        news = row[1]
        #print('전:', news)
        for word in MY_STOP:
            mask = ' ' + word + ' '
            news = news.replace(mask, ' ')
        #print('후:', news)

        articles.append(news)
print('처리한 기사 숫자:', len(articles))

['category', 'text']
처리한 기사 숫자: 2225


In [17]:
#샘플기사 원본 출력

print('샘플 기사 원본')
print(original[MY_SAMPLE])
print(labels[MY_SAMPLE])
print(type(original[MY_SAMPLE]))
print('총 단어 수:', len(original[MY_SAMPLE].split()))

샘플 기사 원본
jack cunningham to stand down veteran labour mp and former cabinet minister jack cunningham has said he will stand down at the next election.  one of the few blair-era ministers to serve under jim callaghan  he was given the agriculture portfolio when labour regained power in 1997. mr cunningham went on to become tony blair s  cabinet enforcer . he has represented the constituency now known as copeland since 1970. mr blair said he was a  huge figure  in labour and a  valued  personal friend .  during labour s long period in opposition  mr cunningham held a number of shadow roles including foreign affairs  the environment and as trade spokesman. as agriculture minister he caused controversy when he decided to ban beef on the bone in the wake of fears over bse. he quit the government in 1999 and in recent years has served as the chairman of the all-party committee on lords reform and has been a loyal supporter of the government from the backbenches.
politics
<class 'str'>
총 단어 수

In [19]:
#7번 블럭
#제외어 삭제 기사 출력
print('제외어 삭제본')
print(articles[MY_SAMPLE])
print('총 단어 수:', len(articles[MY_SAMPLE].split()))
print(type(articles[MY_SAMPLE]))



제외어 삭제본
jack cunningham stand veteran labour mp former cabinet minister jack cunningham said stand next election.  one blair-era ministers serve jim callaghan  given agriculture portfolio labour regained power 1997. mr cunningham went become tony blair  cabinet enforcer . represented constituency known copeland since 1970. mr blair said  huge figure  labour  valued  personal friend .  labour long period opposition  mr cunningham held number shadow roles including foreign affairs  environment trade spokesman. agriculture minister caused controversy decided ban beef bone wake fears bse. quit government 1999 recent years served chairman all-party committee lords reform loyal supporter government backbenches.
총 단어 수: 95
<class 'str'>


In [20]:
for i in range(len(articles)):
    if ' the ' in articles[i]:
        print('원본: ', original[i])
        print('처리: ', articles[i])

원본:  goldsmith:  i was not leant on  the attorney general has again denied being  leant on  by downing street to make the legal case for invading iraq.  claims a written answer on the legality of the war was drafted by downing street were  wholly unfounded   he insisted during stormy lords exchanges. lord goldsmith said the answer represented his  genuinely held independent view  the war was legal. the text was released on the eve of a crucial commons vote in which mps backed the invasion of iraq. many labour mps have since indicated that the attorney general s answer played a pivotal role in their willingness to back the conflict. the government has resisted calls to publish the full advice  saying such papers are always kept confidential. in the house of lords  the attorney general faced a call by former tory lord chancellor lord mackay to now publish the  full text  of the advice - the suggestion was rejected. another peer meanwhile  lord skidelsky  said not to publish the full lega

In [23]:
#기사 Tokenization 처리 

A_token = Tokenizer(num_words=MY_VOCAB,
                    oov_token='OOV')
print(A_token.word_index)
A_token.fit_on_texts(articles)
print(A_token.word_index)



sauber': 28611, 'hosiery': 28612, 'deodorant': 28613, 'mister': 28614, '449': 28615, 'owing': 28616, 'swelled': 28617, 'babacan': 28618, 'accommodated': 28619, 'substitutions': 28620, 'sniffer': 28621, 'guti': 28622, 'ser': 28623, 'nihat': 28624, 'kahveci': 28625, 'albacete': 28626, 'separatist': 28627, 'accosiation': 28628, 'shamil': 28629, 'fabrication': 28630, 'raps': 28631, 'overcharged': 28632, 'departing': 28633, 'hassall': 28634, 'illston': 28635, 'evidentiary': 28636, 'admissibility': 28637, 'vima': 28638, 'childish': 28639, '짙49m': 28640, 'paion': 28641, 'clotting': 28642, 'cursed': 28643, 'werewolf': 28644, 'ranger': 28645, 'cheerleader': 28646, 'elise': 28647, 'philandering': 28648, 'madea': 28649, 'powerpoint': 28650, 'telemedicine': 28651, 'personalized': 28652, 'hatched': 28653, 'designate': 28654, 'prassana': 28655, 'rambathla': 28656, 'choke': 28657, 'foolproof': 28658, 'starbuck': 28659, 'intermittency': 28660, 'pctvts': 28661, 'insect': 28662, 'postulating': 28663, 'n